In [4]:
from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json
import numpy as np
import os
import cv2
import operator

# load json and create model
json_file = open('model-bw.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model-bw.h5")
print("Loaded model from disk")
video = cv2.VideoCapture(0)
top, right, bottom, left = 10, 100, 560, 590


while True :
    _,frame = video.read()
    frame = cv2.flip(frame,1)
    cv2.imshow("video",frame)
    h,w = frame.shape[:2]
    roi = frame[top:bottom, right:left]
    gray = cv2.cvtColor(roi,cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray,(37,37),0)
    #cv2.imshow("gray",gray)
    #cv2.imshow("blur",blur)
    cv2.imshow("ROI",roi)
    test, test_image = cv2.threshold(blur, 120, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    cv2.imshow("test_image",test_image)
    
    contours,hierarchy = cv2.findContours(test_image,cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[-2:]
    if len(contours) > 0 :
        contour = max(contours,key = cv2.contourArea)
        
        if(cv2.contourArea(contour) > 2000):
            x, y, w, h = cv2.boundingRect(contour)
            newImage = test_image[y:y + h, x:x + w]
    
    newImage = cv2.resize(newImage, (28, 28))
    newImage = np.array(newImage)
    newImage = newImage.flatten()
    newImage = newImage.reshape(1,28,28, 1)
    
    result = loaded_model.predict(newImage)
    
    prediction = {'ZERO': result[0][0], 
                  'ONE': result[0][1], 
                  'TWO': result[0][2],
                  'THREE': result[0][3],
                  'FOUR': result[0][4],
                  'SIX': result[0][6],
                  'SEVEN': result[0][7],
                  'EIGHT': result[0][8],
                  'NINE': result[0][9],
                  'FIVE': result[0][5]}
    # Sorting based on top prediction
    prediction = sorted(prediction.items(), key=operator.itemgetter(1), reverse=True)
    #print(result)
    
    cv2.putText(frame,prediction[0][0],(10,380),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)   
    #cv2.putText(img, "Deep Network :  " + str(ans3), (10, 380),
                    #cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)        
    cv2.imshow("frame",frame)
    interrupt = cv2.waitKey(10)
    if interrupt & 0xFF == 27: # esc key
        break

        
video.release()
cv2.destroyAllWindows()

Loaded model from disk
